# Transactions

- one or more statements, 
- When executing, all or none of the statements are executed (Either all or None)
- Example:
    - Purpose : Transfer $100 from account A to account B
    - Operation 1 : Subtract $100 from account A
    - Operation 2 : Add $100 to account B
    - Operation 2 FAILS (We could not Add $100 to account B )
    - Then Operation 1 reverts (UNDO Subtract $100 from account A)

# Transaction workflow

- `BEGIN TRAN... COMMIT TRAN... ROLLBACK TRAN`
- `BEGIN TRAN` : Begin a trnsaction
- `COMMIT TRAN` : Transaction was successful
- `ROLLBACK TRAN` : Revert the transaction in case of failure
- Generally used with `TRY..CATCH`
- example: 
```
BEGIN TRY
    BEGIN TRAN;
        ....transaction query....
    COMMIT TRAN;
END TRY
BEGIN CATCH
    ROLLBACK TRAN;
END CATCH
```

- SEE MORE OPTIONS IN THE SLIDES

# `@@TRANCOUNT`

- Returns Number of active `BEGIN TRAN` statements in your current connection
    - greater than 0 -> open transaction
    - 0 -> no open transaction
- The value is modified by different `TRAN` workflow
    - `BEGIN TRAN` -> `@@TRANCOUNT + 1`
    - `COMMIT TRAN` -> `@@TRANCOUNT - 1`
    - `ROLLBACK TRAN` -> `@@TRANCOUNT = 0` 
        - will not be 0 if there is a savepoint
- example: `SELECT @@TRANCOUNT`

- example:

```
BEGIN TRY
    BEGIN TRAN;
        ...query...
    IF (@@TRANCOUNT > 0)
        COMMIT TRAN;
END TRY
BEGIN CATCH
    IF (@@TRANCOUNT > 0)
        ROLLBACK TRAN;
END CATCH
```

# Savepoints

- Saving a state inside transaction
- with `SAVE TRAN savepoint_name` keywords
- Rollback to a savepoint : `ROLLBACK TRAN savepoint_name`
- example:
    ```
    BEGIN TRAN; 
    SAVE TRAN savepoint1;
        ...query1..; 
    SAVE TRAN savepoint2;
        ...query2..; 
    ROLLBACK TRAN savepoint2;
    ROLLBACK TRAN savepoint1; 
    SAVE TRAN savepoint3;
    COMMIT TRAN;
    ```

# `XACT_ABORT`

- current transaction will be automatically rolled back when an error occurs.
- settings : `SET XACT_ABORT { ON | OFF }`
- Default settings : `SET XACT_ABORT OFF`
    - If there is an error: There can be open transactions
- Turn on : `SET XACT_ABORT ON`
    - Rollbacks the transaction and aborts the execution
    - Using with `RAISEERROR` will still make changes to the database
    - Using with `THROW` will `ROLLBACK` changes to the database (SAFE)

# `XACT_STATE()`

- With 
- 0 -> no open transaction
- 1 -> open and committable transaction
- -1 -> open and uncommittable transactions (Doomed transactions)
    - can't commit
    - can't rollback to a savepoint
    - can rollback the full transaction
    - can't make any changes/can read data
    - RESOLVE : ROLLBACK FULL TRANSACTION: `ROLLBACK TRAN;`
- example:
    ```
    SET XACT_ABORT ON;
    BEGIN TRY
        BEGIN TRAN;
            ...query....
        COMMIT TRAN;
    END TRY
    BEGIN CATCH
        IF XACT_STATE() = -1
            ROLLBACK TRAN;
        IF XACT_STATE() = 1
            COMMIT TRAN;
        SELECT ERROR_MESSAGE() AS Error_message;
    END CATCH
    ```

# TIPS

- See affected rowcount with `@@ROWCOUNT`
```
IF @@ROWCOUNT > 200 
	BEGIN 
		SELECT 'Affected rows in the query'; 
	END
```